In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from scipy.interpolate import griddata
import os
import pandas as pd
from scipy.sparse import lil_matrix, csr_matrix
from scipy.sparse.linalg import lsqr
import time

# Physical Parameters
C1 = 1.0    # Nonlinear term coefficient (u·∇u)
C2 = 0.01   # Viscosity coefficient (ν)

def add_noise(data, noise_level=0.0):
    """Add component-wise Gaussian noise"""
    if noise_level == 0.0:
        return data.copy()
    noisy_data = data.copy()
    for i in range(data.shape[1]):
        std = np.std(data[:, i])
        noisy_data[:, i] = data[:, i] + noise_level * std * np.random.randn(data.shape[0])
    return noisy_data

def load_data(mat_path, target_t=1.0):
    """Load and prepare cylinder wake data at specific time"""
    if not os.path.exists(mat_path):
        raise FileNotFoundError(f"Data file not found at: {mat_path}")
    data = loadmat(mat_path)
    X_star = data['X_star']
    U_star = data['U_star']
    p_star = data['p_star']
    t_all = data['t'].flatten()
    t_idx = np.argmin(np.abs(t_all - target_t))
    x, y = X_star[:, 0], X_star[:, 1]
    u, v = U_star[:, 0], U_star[:, 1]
    p = p_star[:, t_idx].flatten()
    T = np.full_like(x, target_t)
    return np.column_stack((x, y, T)), np.column_stack((u, v, p)), x, y, p

def preprocess_data(X, Y):
    """Normalize data using StandardScaler"""
    scaler = StandardScaler()
    Y_norm = scaler.fit_transform(Y)
    return Y_norm, scaler

def train_mlp(X, Y_norm):
    """Train MLP regressor with tanh activation"""
    model = MLPRegressor(hidden_layer_sizes=(60, 40), activation='tanh', solver='adam',
                        max_iter=1000, tol=1e-6, random_state=42)
    model.fit(X, Y_norm)
    return model

def estimate_parameters(x, y, u, v, p):
    """Physics-informed parameter estimation"""
    grid_size = 150
    x_lin = np.linspace(-2, 10, grid_size)
    y_lin = np.linspace(-4, 4, grid_size)
    Xg, Yg = np.meshgrid(x_lin, y_lin)
    pts = np.column_stack((x, y))

    # Interpolate with masking near cylinder
    Zu = griddata(pts, u, (Xg, Yg), method='linear')
    Zv = griddata(pts, v, (Xg, Yg), method='linear')
    Zp = griddata(pts, p, (Xg, Yg), method='linear')
    cylinder_mask = (Xg**2 + Yg**2) < 0.25
    Zu[cylinder_mask] = np.nan
    Zv[cylinder_mask] = np.nan
    Zp[cylinder_mask] = np.nan

    # Compute derivatives
    dx = x_lin[1] - x_lin[0]
    dy = y_lin[1] - y_lin[0]
    dudx, dudy = np.gradient(Zu, dx, dy)
    dvdx, dvdy = np.gradient(Zv, dx, dy)
    dpdx, dpdy = np.gradient(Zp, dx, dy)
    d2udx2 = np.gradient(dudx, dx, axis=1)
    d2udy2 = np.gradient(dudy, dy, axis=0)
    d2vdx2 = np.gradient(dvdx, dx, axis=1)
    d2vdy2 = np.gradient(dvdy, dy, axis=0)

    # Filter valid points
    valid_mask = ~np.isnan(Zu.ravel())
    u_flat = Zu.ravel()[valid_mask]
    v_flat = Zv.ravel()[valid_mask]

    # Build linear system
    num_points = len(u_flat)
    A = lil_matrix((2*num_points, 2))
    b = np.zeros(2*num_points)

    # Formulate Navier-Stokes equations
    conv_u = u_flat * dudx.ravel()[valid_mask] + v_flat * dudy.ravel()[valid_mask]
    diff_u = d2udx2.ravel()[valid_mask] + d2udy2.ravel()[valid_mask]
    conv_v = u_flat * dvdx.ravel()[valid_mask] + v_flat * dvdy.ravel()[valid_mask]
    diff_v = d2vdx2.ravel()[valid_mask] + d2vdy2.ravel()[valid_mask]

    A[:num_points, 0] = conv_u.reshape(-1, 1)
    A[:num_points, 1] = -diff_u.reshape(-1, 1)
    b[:num_points] = -dpdx.ravel()[valid_mask]

    A[num_points:, 0] = conv_v.reshape(-1, 1)
    A[num_points:, 1] = -diff_v.reshape(-1, 1)
    b[num_points:] = -dpdy.ravel()[valid_mask]

    # Solve sparse system
    result = lsqr(csr_matrix(A), b, atol=1e-4, btol=1e-4)
    return result[0], result[4]

def plot_velocity_comparison(x, y, u_pred, v_pred, u_true, v_true, title, output_dir, noise_level):
    """Create comparison plots with consistent color scaling"""
    grid_size = 300
    x_lin = np.linspace(-2, 10, grid_size)
    y_lin = np.linspace(-4, 4, grid_size)
    Xg, Yg = np.meshgrid(x_lin, y_lin)
    pts = np.column_stack((x, y))

    # Interpolate with common scale
    Zu_pred = griddata(pts, u_pred, (Xg, Yg), method='linear')
    Zv_pred = griddata(pts, v_pred, (Xg, Yg), method='linear')
    Zu_true = griddata(pts, u_true, (Xg, Yg), method='linear')
    Zv_true = griddata(pts, v_true, (Xg, Yg), method='linear')

    # Use symmetric scaling based on true data
    max_val = max(np.nanmax(np.abs(Zu_true)), np.nanmax(np.abs(Zv_true)))
    vmin, vmax = -max_val, max_val

    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # Predicted velocities
    im1 = axes[0,0].imshow(Zu_pred, extent=(-2,10,-4,4), cmap='coolwarm',
                          vmin=vmin, vmax=vmax, origin='lower', aspect='auto')
    axes[0,0].set_title('Predicted u-velocity')
    fig.colorbar(im1, ax=axes[0,0])
    axes[0,0].add_patch(plt.Circle((0, 0), 0.5, color='gray', fill=True))

    im2 = axes[0,1].imshow(Zv_pred, extent=(-2,10,-4,4), cmap='coolwarm',
                          vmin=vmin, vmax=vmax, origin='lower', aspect='auto')
    axes[0,1].set_title('Predicted v-velocity')
    fig.colorbar(im2, ax=axes[0,1])
    axes[0,1].add_patch(plt.Circle((0, 0), 0.5, color='gray', fill=True))

    # True velocities
    im3 = axes[1,0].imshow(Zu_true, extent=(-2,10,-4,4), cmap='coolwarm',
                          vmin=vmin, vmax=vmax, origin='lower', aspect='auto')
    axes[1,0].set_title('True u-velocity')
    fig.colorbar(im3, ax=axes[1,0])
    axes[1,0].add_patch(plt.Circle((0, 0), 0.5, color='gray', fill=True))

    im4 = axes[1,1].imshow(Zv_true, extent=(-2,10,-4,4), cmap='coolwarm',
                          vmin=vmin, vmax=vmax, origin='lower', aspect='auto')
    axes[1,1].set_title('True v-velocity')
    fig.colorbar(im4, ax=axes[1,1])
    axes[1,1].add_patch(plt.Circle((0, 0), 0.5, color='gray', fill=True))

    plt.suptitle(title)
    plt.tight_layout()
    fig.savefig(os.path.join(output_dir, f'velocity_comparison_{noise_level}.png'),
               dpi=300, bbox_inches='tight')
    plt.close(fig)

def plot_pressure_comparison(x, y, p_true, p_pred, title, output_dir, noise_level):
    """Create pressure comparison plot"""
    grid_size = 200
    x_lin = np.linspace(-2, 10, grid_size)
    y_lin = np.linspace(-4, 4, grid_size)
    Xg, Yg = np.meshgrid(x_lin, y_lin)
    pts = np.column_stack((x, y))

    Z_pred = griddata(pts, p_pred, (Xg, Yg), method='linear')
    Z_true = griddata(pts, p_true, (Xg, Yg), method='linear')

    # Center pressure by removing mean difference
    pressure_diff = np.nanmean(Z_true) - np.nanmean(Z_pred)
    Z_pred += pressure_diff

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Predicted pressure
    pcm1 = ax1.contourf(Xg, Yg, Z_pred, levels=50, cmap='RdYlBu')
    ax1.add_patch(plt.Circle((0, 0), 0.5, color='gray', fill=True))
    ax1.set_title('Predicted Pressure')
    ax1.set_aspect('equal')
    fig.colorbar(pcm1, ax=ax1)

    # True pressure
    pcm2 = ax2.contourf(Xg, Yg, Z_true, levels=50, cmap='RdYlBu')
    ax2.add_patch(plt.Circle((0, 0), 0.5, color='gray', fill=True))
    ax2.set_title('True Pressure')
    ax2.set_aspect('equal')
    fig.colorbar(pcm2, ax=ax2)

    plt.suptitle(title)
    plt.tight_layout()
    fig.savefig(os.path.join(output_dir, f'pressure_comparison_{noise_level}.png'),
               dpi=300, bbox_inches='tight')
    plt.close(fig)

def calculate_metrics(u_true, v_true, p_true, u_pred, v_pred, p_pred):
    """Calculate quantitative performance metrics"""
    metrics = {}

    # Velocity errors
    metrics['u_mae'] = np.mean(np.abs(u_true - u_pred))
    metrics['u_rmse'] = np.sqrt(np.mean((u_true - u_pred)**2))
    metrics['u_max'] = np.max(np.abs(u_true))

    metrics['v_mae'] = np.mean(np.abs(v_true - v_pred))
    metrics['v_rmse'] = np.sqrt(np.mean((v_true - v_pred)**2))
    metrics['v_max'] = np.max(np.abs(v_true))

    # Pressure errors (considering constant offset)
    p_offset = np.mean(p_true) - np.mean(p_pred)
    p_pred_corr = p_pred + p_offset
    metrics['p_mae'] = np.mean(np.abs(p_true - p_pred_corr))
    metrics['p_rmse'] = np.sqrt(np.mean((p_true - p_pred_corr)**2))

    return metrics

def generate_results_table(results):
    """Generate publication-quality results table"""
    latex = "\\begin{table}[ht]\n"
    latex += "\\centering\n"
    latex += "\\caption{Parameter estimation results}\n"
    latex += "\\begin{tabular}{|c|c|c|c|c|c|}\n"
    latex += "\\hline\n"
    latex += "Noise Level & C1 Estimate & C2 Estimate & u-RMSE & v-RMSE & p-RMSE \\\\\n"
    latex += "\\hline\n"

    for res in results:
        latex += f"{res['Noise Level']*100:.1f}\% & {res['C1_estimated']:.5f} & {res['C2_estimated']:.5f} & "
        latex += f"{res['u_RMSE']:.2e} & {res['v_RMSE']:.2e} & {res['p_RMSE']:.2e} \\\\\n"
        latex += "\\hline\n"

    latex += "\\end{tabular}\n"
    latex += "\\end{table}"

    # Add PDE equations
    clean_data = next(r for r in results if r['Noise Level'] == 0.0)
    noise_data = next(r for r in results if r['Noise Level'] == 0.01)

    latex += "\n\n\\begin{align*}\n"
    latex += "\\text{Correct PDE:} & \\quad u_t + (uu_x + vu_y) = -p_x + 0.01(u_{xx} + u_{yy}) \\\\\n"
    latex += "& \\quad v_t + (uv_x + vv_y) = -p_y + 0.01(v_{xx} + v_{yy}) \\\\[10pt]\n"
    latex += f"\\text{{Identified (clean):}} & \\quad u_t + {clean_data['C1_estimated']:.3f}(uu_x + vu_y) = -p_x + {clean_data['C2_estimated']:.5f}(u_{{xx}} + u_{{yy}}) \\\\\n"
    latex += f"& \\quad v_t + {clean_data['C1_estimated']:.3f}(uv_x + vv_y) = -p_y + {clean_data['C2_estimated']:.5f}(v_{{xx}} + v_{{yy}}) \\\\[10pt]\n"
    latex += f"\\text{{Identified (1\\% noise):}} & \\quad u_t + {noise_data['C1_estimated']:.3f}(uu_x + vu_y) = -p_x + {noise_data['C2_estimated']:.5f}(u_{{xx}} + u_{{yy}}) \\\\\n"
    latex += f"& \\quad v_t + {noise_data['C1_estimated']:.3f}(uv_x + vv_y) = -p_y + {noise_data['C2_estimated']:.5f}(v_{{xx}} + v_{{yy}})\n"
    latex += "\\end{align*}"

    return latex

def main():
    # Create output directory on Desktop
    output_dir = os.path.join(os.path.expanduser('~'), 'Desktop', 'NextStep_Results')
    os.makedirs(output_dir, exist_ok=True)

    # Use your exact file path
    mat_path = r'C:\Users\sauja\Downloads\cylinder_nektar_wake.mat'

    # Verify file exists
    if not os.path.exists(mat_path):
        raise FileNotFoundError(
            f"Data file not found at: {mat_path}\n"
            "Please verify:\n"
            "1. The file exists at this location\n"
            "2. The filename is EXACTLY 'cylinder_nektar_wake.mat'\n"
            "3. You have read permissions"
        )

    # Load data
    X, Y, x, y, p_true = load_data(mat_path, target_t=1.0)
    u_true, v_true = Y[:, 0], Y[:, 1]

    # Test multiple noise levels
    noise_levels = [0.0, 0.01, 0.05, 0.1, 0.5]
    results = []

    for noise_level in noise_levels:
        start_time = time.time()
        print(f"Processing noise level: {noise_level*100:.1f}%")

        # Add noise only to velocities
        Y_noisy = Y.copy()
        Y_noisy[:, 0] = add_noise(Y[:, 0:1], noise_level).flatten()
        Y_noisy[:, 1] = add_noise(Y[:, 1:2], noise_level).flatten()

        # Preprocess and train
        Y_norm, scaler = preprocess_data(X, Y_noisy)
        model = train_mlp(X, Y_norm)

        # Predict and denormalize
        Y_pred = scaler.inverse_transform(model.predict(X))
        u_pred, v_pred, p_pred = Y_pred[:, 0], Y_pred[:, 1], Y_pred[:, 2]

        # Estimate parameters
        try:
            params, residual_norm = estimate_parameters(x, y, u_pred, v_pred, p_pred)
            c1_est, c2_est = params
        except Exception as e:
            print(f"Parameter estimation failed: {str(e)}")
            c1_est, c2_est = np.nan, np.nan
            residual_norm = np.nan

        # Calculate metrics
        metrics = calculate_metrics(u_true, v_true, p_true, u_pred, v_pred, p_pred)

        # Store results
        results.append({
            'Noise Level': noise_level,
            'C1_estimated': c1_est,
            'C2_estimated': c2_est,
            'u_MAE': metrics['u_mae'],
            'u_RMSE': metrics['u_rmse'],
            'v_MAE': metrics['v_mae'],
            'v_RMSE': metrics['v_rmse'],
            'p_MAE': metrics['p_mae'],
            'p_RMSE': metrics['p_rmse'],
            'Residual': residual_norm
        })

        # Save plots for key noise levels
        if noise_level in [0.0, 0.01, 0.1]:
            plot_velocity_comparison(
                x, y, u_pred, v_pred, u_true, v_true,
                f"Cylinder Wake at t=1.0 | Noise={noise_level*100:.1f}%",
                output_dir, noise_level
            )

            plot_pressure_comparison(
                x, y, p_true, p_pred,
                f"Cylinder Wake at t=1.0 | Noise={noise_level*100:.1f}%",
                output_dir, noise_level
            )

        print(f"Completed in {time.time()-start_time:.1f} seconds")

    # Save results
    results_df = pd.DataFrame(results)
    results_csv = os.path.join(output_dir, 'parameter_results.csv')
    results_df.to_csv(results_csv, index=False)

    # Generate LaTeX table
    latex_table = generate_results_table(results)
    with open(os.path.join(output_dir, 'results_table.tex'), 'w') as f:
        f.write(latex_table)

    # Print summary
    print("\nResults Summary:")
    print(results_df[['Noise Level', 'C1_estimated', 'C2_estimated',
                     'u_RMSE', 'v_RMSE', 'p_RMSE']].to_string())

    print(f"\nResults saved to: {output_dir}")
    print(f"CSV file: {results_csv}")
    print(f"LaTeX table: {os.path.join(output_dir, 'results_table.tex')}")

if __name__ == "__main__":
    main()